## Homework 09:  Parallel Programming 02

## Due Date: Apr 19, 2023, 11:59pm

#### Firstname Lastname: Giulio Duregon

#### E-mail: gjd9961@nyu.edu

#### Enter your solutions and submit this notebook

---

**Problem 1 (40p)**

In this problem the goal is to calculate the mean and standard deviation of a large list of numbers by using Reduction as one of Collective Operations, see Lecture 11. 


Consider $N = 256000$ random variables uniform on $[0, 1]$, call these $x_0, x_1, \dots, x_{N - 1}$.  


Write an MPI program with $N=16$ processes that outputs the average and standard deviation of $x_0, x_1, \dots, x_{N - 1}$.


To simplify the problem, let one process, say `Process 0`, independently draws $N$ samples uniformly on $[0, 1]$.

How do you explain the results?


In [138]:
%%writefile 2020_spring_sol09_pr01.py
import numpy as np
from mpi4py import MPI
from time import time

# Standard Setup for MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
root = 0


# Helper function for computing averages
def chunk_average(points, n):
    return np.divide(np.sum(points),n)

def chunk_std(points, mean, n):
    return np.divide(np.sum(np.power(np.subtract(points, mean),2)),n)

# Number of samples to draw
n = 256_000

# Compute partition
num_points_per_process = (n // size)

# Let Process 0 Draw Samples, and hold final values
if rank == 0:
    start_time = time()
    samples = np.random.uniform(0,1,n)
    target_average = np.zeros(1, dtype=float)
    target_std = np.zeros(1, dtype=float)
else:
    # Need to be initialized to avoid errors
    samples = None
    target_average = np.zeros(1,dtype=float)
    target_std = np.zeros(1,dtype=float)

# Temp buffers for all processes
helper_average = np.zeros(1, dtype=float)
helper_std = np.zeros(1, dtype=float)

# Provide buffer for all samples
small_samples = np.zeros(num_points_per_process, dtype=float)

# Scatter the points
comm.Scatter(samples, small_samples, root=root)

#Perform Average calculation on each chunk (N is Global here)
helper_average = chunk_average(small_samples, n)

# Reduce The Helper averages -> Target Average (True avg of all points)
comm.Reduce(helper_average, target_average, op=MPI.SUM, root=root)

# Force Synchronization before sending the average back out
# (Used in std calculation)

comm.Bcast(target_average, root=root)

# # Perform standard deviation calcs on chunks
helper_std = chunk_std(small_samples, target_average, n)

# Send back and reduce into a sum
comm.Reduce(helper_std, target_std, op=MPI.SUM, root=root)

if rank == 0:
    target_std = np.sqrt(target_std)
    
if rank == 0:
    print(f"Time Elapsed: {time()-start_time}s")
    print(f"target_average={target_average}")
    print(f"target_std={target_std}")

Overwriting 2020_spring_sol09_pr01.py


In [66]:
!mpirun -n 16 --oversubscribe python 2020_spring_sol09_pr01.py

Time Elapsed: 0.012256860733032227s
target_average=[0.5004913]
target_std=[0.28894069]
--------------------------------------------------------------------------
A system call failed during shared memory initialization that should
not have.  It is likely that your MPI job will now either abort or
experience performance degradation.

  Local host:  Giulios-MBP.lan
  System call: unlink(2) /var/folders/rk/rwsr6gss0vz3g4fz3_kt0x0m0000gn/T//ompi.Giulios-MBP.501/pid.3718/1/vader_segment.Giulios-MBP.501.5c7d0001.11
  Error:       No such file or directory (errno 2)
--------------------------------------------------------------------------



---
**Problem 2 (60p)**

In this problem the goal is to demonstrate how one can use a Domain Decomposition and  Collective Operations. 

Consider the exponential distribution $X \sim \textrm{Exp}(1)$ with the unit mean. Find numerical approximations of moments of the exponential random variable. 

That is, for a random variable $X$ with the distribution $f(x) = e^{-x}$ for $x \geq 0$, compute the first $15$ moments, where the $k$-th moment is defined as:
$$I_k = \int_{0}^{\infty} x^k f(x) dx.$$


Your program should use MPI parallel collective instructions, where the integration is performed in parallel over $N=16$ processes, over a finite range $[0, M)$, where $M=1000$, with $N = 16$ partitions and $1000$ increments per partition, see Lecture 10 and 11.

Provide evaluations of $J_1, J_2, \dots, J_{15}$, where $$J_k = \int_{0}^{M} x^k f(x) dx.$$


In [171]:
%%writefile 2020_spring_sol09_pr02.py
import numpy as np
from mpi4py import MPI

# Standard Setup for MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
root = 0

# Define helper function for integral calculation
def integral(start, stop, moment, inc_per_partition=1000):
    partial_int = np.zeros(1, dtype=float)
    step_size = (stop - start)/ inc_per_partition
    x = start
    for _ in range(inc_per_partition):
        partial_int += np.multiply(np.power(x,moment), np.exp(-x))
        x += step_size
    return np.multiply(partial_int, step_size)
        

start, stop = 0, 1000
temp_start = 0
N = 16
range_per_process = (stop - start) // N
inc_per_partition = 1000

worker_res = np.zeros(1)

if rank == 0:
    work_chunks = []
    for _ in range(16):
        work_chunks.append((temp_start, temp_start + range_per_process))
        temp_start += range_per_process
    work_chunks = np.array(work_chunks, dtype=float)
    moment_results = np.zeros(16,dtype=float)
    moment_chunk = np.zeros(2, dtype=float)
    gather_res = np.zeros(1,dtype=float)
else:
    work_chunks = None
    moment_results = None
    gather_res = np.zeros(1,dtype=float)
    moment_chunk = np.zeros(2, dtype=float)
    
# Scatter the bits of work to do
comm.Scatter(work_chunks, moment_chunk, root=root)

for k in range(16):
    # Perform integration for ith moment
    worker_res[0] = integral(moment_chunk[0], moment_chunk[1], k+1)
    comm.Reduce(worker_res, gather_res, MPI.SUM, root)
    comm.Barrier()
    if rank == 0:
        moment_results[k] = gather_res[0]
        
if rank == 0:
    ground_truth_arr = []
    ground_truth = 1
    for i in range(1,17):
        ground_truth *= i
        ground_truth_arr.append(ground_truth)
    ground_truth_arr = np.array(ground_truth_arr, dtype=float)
    
    for i in range(0,16):
        print(f"Moment {i+1} result = {moment_results[i]}, Ground Truth: {ground_truth_arr[i]} Error: {np.abs(ground_truth_arr[i]-moment_results[i])}")

Overwriting 2020_spring_sol09_pr02.py


In [154]:
!mpirun -n 16 --oversubscribe python 2020_spring_sol09_pr02.py

**Bonus Question (10 points):** 

What is the value of $I_k$, as a function of $k$? How can it be derived?

### Bonus Question

We have by definition that the $k$-th moment of the exponential distribution is:

$$I_k = \int_{0}^{\infty} x^k e^{-x} dx.$$

We can integrate by parts, to show that: 

$$I_k = k I_{k-1},$$

where $I_0 = 1$.

The proof is as follows, by starting with the aforementioned definition:
$$I_k = \int_{0}^{\infty} x^k e^{-x} dx.$$

Integrating by parts (with $u = x^k$) and ($dv = e^{-x} dx$) gives:

$$I_k = -x^k e^{-x} \bigg\rvert_{0}^{\infty} + k \int_{0}^{\infty} x^{k-1} e^{-x} dx.$$

Since $x^k e^{-x} \rightarrow 0$ as $x \rightarrow \infty$, the first term on the right-hand side is zero. Thus, we have:

$$I_k = k \int_{0}^{\infty} x^{k-1} e^{-x} dx = k I_{k-1}$$ 

Thus completing the proof
